In [1]:
import pandas as pd
import requests as r
from collections import Counter
import json
import re
import collections

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

In [3]:
def removeRegex(stc):
    stc = stc.replace('(','')
    stc = stc.replace(')','')
    return stc

In [4]:
def nested_dict():
    return collections.defaultdict(nested_dict)

In [5]:
FORMS_URL = "http://unep.localhost/json/unep-forms-with-variables.json"
COUNTRY_QUESTION = "Country"
COUNTRY_FILTER = "Member State"
DATA_SOURCE = "data-production_2020-03-16.csv"

In [6]:
def generate_code(x,y,t):
    for i, a in enumerate(x):
        code = y +'#' + str(i)
        if t == COUNTRY_QUESTION:
            a.update({
                'code':a['v']
            })
        else:
            a.update({
                'code':code
            })
    return x

In [7]:
dataframe = pd.read_csv('./source/' + DATA_SOURCE)
forms = pd.DataFrame(r.get(FORMS_URL).json())
forms['code'] = forms['q'].apply(lambda x:x.split(' ')[0].replace('.','').upper())
forms['a'] = forms.apply(lambda x: generate_code(x['a'], x['code'], x['s']), axis=1)
col_names = forms[['q','code']].to_dict('records')
col_names = {q['q']:q['code'] for q in col_names}

In [8]:
questions = forms['code'].to_list()
dataframe = dataframe.rename(columns=col_names)

In [9]:
#DEBUG
#dataframe = dataframe.loc[0:1]

In [10]:
def collect(sentence, options):
    data = []
    if sentence == sentence:
        sentence = removeRegex(sentence)
        for option in options:
            option_value = removeRegex(option['v'])
            if re.findall(option_value, str(sentence)):
                data.append(option['code'])
    return data

In [11]:
country_qcode = forms[forms['s'] == COUNTRY_QUESTION].reset_index()
country_qcode = country_qcode['code'][0]

In [12]:
metadatas = ['Unique Response Number','StartDate','CompletionDate']
questionlist = []
dataset = []
for index, row in dataframe.iterrows():
    cdata = {}
    for mdata in metadatas:
        cdata.update({mdata: row[mdata]})
    for qdata in list(dataframe):
        for q in questions:
            if qdata == q:
                qst = forms[forms['code'] == q]
                qtype = qst['t'].values[0]
                qopt = qst['a'].values[0]
                if qtype == "OPTIONS":
                    cdata.update({q: collect(row[q], qopt)})
                    questionlist.append(q)
                else:
                    cdata.update({q: row[q]})
    dataset.append(cdata)

In [13]:
baked = pd.DataFrame(dataset)

In [30]:
baked[country_qcode] = baked[country_qcode].apply("##".join)

,index,Unique Response Number,StartDate,CompletionDate,4D,5,5A,5B,6,6A,7,8,9,9A,9AI,9B,9BI,9BII,9BIIA,9BIII,9BIIIA,9BIIIB,9BIIIBI,9C,9CI,9CII,9CIIA,9D,9DI,9DII,9DIII,9DIV,9DV,9DVI,9DVII,9DVIII,9DVIIIA,9DVIIIB,10,11,11A,12,12A,12B,12BI,12C,12CI,12D,12DI,13,13A,13AI,13B,13BI,14,14A,15,15A,16,16A,17,17A,18,18A,19,19A,20,20A,21,21A,21B,21BI,21C,21CI,22,22A,23,24,25,26,27,27A,27B,28,28A,29,29A,29B,29C,29D,29E,30
0,0,545728-545719-53376637,2019-12-20 17:36:36 UTC,2019-12-30 15:42:40 UTC,The SeaCleaners,On behalf of an organisation,NaN,The SeaCleaners,Yes,4.0,AWARENESS RAISING,"-awareness raising activities in schools, trad...",[9#1],[],NaN,"[9B#0, 9B#1, 9B#2, 9B#3]",NaN,"[9BII#0, 9BII#1, 9BII#2, 9BII#3, 9BII#4]",NaN,"[9BIII#1, 9BIII#4]",NaN,[],NaN,[],NaN,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],[11#2],NaN,[12#2],NaN,[],NaN,[],NaN,[12D#0],NaN,[13#1],[],NaN,[13B#2],NaN,[14#1],NaN,[15#2],NaN,France##Germany##Indonesia##Switzerland,NaN,"[17#8, 17#9, 17#10, 17#11]",NaN,"[18#1, 18#2, 18#3, 18#4, 18#5, 18#6, 18#7, 18#8]",NaN,"[19#0, 19#1, 19#2]",NaN,"[20#0, 20#1, 20#2, 20#3, 20#4, 20#5, 20#6]",NaN,[21#0],NaN,[21B#12],NaN,[],NaN,[22#16],NaN,300000.0,euros,500000.0,euros,"[27#1, 27#3]",NaN,private sponsorship,[28#2],NaN,https://www.theseacleaners.org/en/educational-...,NaN,NaN,NaN,NaN,NaN,NaN
1,36,545728-545719-54251922,2020-01-24 10:13:43 UTC,2020-01-24 11:10:11 UTC,Race for Water Foundation,On behalf of an organisation,NaN,Race for Water Foundation,Yes,3.0,SHARE Program,"Alert decision-makers, raise public awareness...",[9#1],[],NaN,"[9B#0, 9B#1, 9B#2, 9B#3]",NaN,"[9BII#0, 9BII#1, 9BII#2, 9BII#3, 9BII#4]",NaN,[9BIII#3],NaN,[],NaN,[],NaN,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[10#0],[11#2],NaN,[12#2],NaN,[],NaN,[],NaN,[12D#0],NaN,[13#0],[13A#1],NaN,[],NaN,[14#1],NaN,[15#0],NaN,Chile##China##Dominican Republic##Egypt##Fiji#...,NaN,[17#13],NaN,"[18#1, 18#2, 18#3, 18#4, 18#5, 18#6, 18#7, 18#8]",NaN,"[19#0, 19#1, 19#2]",NaN,[20#6],NaN,"[21#0, 21#2]",NaN,[21B#12],NaN,[21C#2],NaN,"[22#0, 22#1, 22#8, 22#9, 22#10, 22#11, 22#13, ...",NaN,NaN,NaN,NaN,NaN,"[27#1, 27#2, 27#3, 27#4]",NaN,NaN,[28#3],NaN,https://www.raceforwater.org/en/the-race-for-w...,https://www.raceforwater.org/en/support-us/cit...,https://www.youtube.com/user/multioneattitude/...,NaN,NaN,NaN,At each stopover at our round the world expedi...
2,42,545728-545719-54334516,2020-01-27 10:00:30 UTC,2020-01-27 11:12:43 UTC,Race for Water Foundation,On behalf of an organisation,NaN,Race for Water Foundation,"No, I am returning to the survey to report on ...",NaN,ACT Program : a local value chain for plastic ...,- Give value to mixed plastic waste to incenti...,[9#2],[],NaN,[],NaN,[],NaN,[],NaN,[],NaN,"[9C#2, 9C#3, 9C#6, 9C#9]",NaN,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[10#1],[11#1],NaN,"[12#0, 12#1, 12#2]",NaN,"[12B#0, 12B#1, 12B#2]",NaN,"[12C#0, 12C#1, 12C#2]",NaN,"[12D#0, 12D#1]",NaN,"[13#1, 13#2]",[],NaN,[],NaN,[14#2],Not yet as we are still in the feasbility stud...,[15#0],NaN,Chile##Dominican Republic##France##Malaysia##P...,New Caledonia,"[17#1, 17#2, 17#3, 17#4, 17#5, 17#6, 17#7, 17#8]",NaN,"[18#5, 18#6, 18#7]",NaN,[19#2],NaN,"[20#0, 20#1, 20#2, 20#3, 20#4, 20#5]",NaN,[21#0],NaN,"[21B#0, 21B#1, 21B#2, 21B#3, 21B#4, 21B#5, 21B...",NaN,[],NaN,"[22#0, 22#10, 22#11, 22#12, 22#13, 22#14, 22#15]",NaN,10000000.0,Euro,NaN,NaN,"[27#1, 27#3]",NaN,NaN,[28#3],NaN,https://www.raceforwater.org/en/the-race-for-w...,NaN,NaN,NaN,NaN,NaN,So far most of the money invested (around 60%)...
3,46,545728-545719-54387783,2020-01-28 09:18:19 UTC,2020-01-28 11:05:52 UTC,Race for Water Foundation,On behalf of an organisation,NaN,Race for Water Foundation,"No, I am returning to the survey to report on ...",NaN,LEARN Program,Contribute to the advancement of scientific kn...,[],[],NaN,[],NaN,[],NaN,[],NaN,[],NaN,[],NaN,[],NaN,"[9D#0, 9D#3, 9D#4, 9D#6]",NaN,"During the Race for Water Odyssey 2017-2021, w...",NaN,NaN,During the first Race for Water Odyssey in 201...,"During the Race for

In [15]:
#qobject = {}
#for question in questionlist:
#    if question != country_qcode:
#        dataopt = {}
#        options = forms[forms['code'] == question]['a'].values[0]
#        for option in options:
#            country_value = option['code']
#            dataopt.update({country_value:0})
#        qobject.update({question:dataopt})

In [16]:
#results = nested_dict()
countrylist = forms[forms['code'] == country_qcode]['a'].values[0]
#for country in countrylist:
#    results.update({country['v']:qobject})

In [ ]:
results = nested_dict()

In [ ]:
results['Joy']['Value'] = 50

In [ ]:
len(results)

In [ ]:
for data in baked.to_dict('records'):
    for country in data[country_qcode]:
        for question in questionlist:
            results[country][question]
            if question != country_qcode:
                if question == "9" and country == "Indonesia":
                    print(country, data[question])
        #        for answer in data[question]:
        #            try:
        #                results[country][question][answer] += 1
        #                print(country)
        #            except:
        #                results[country][question][answer] = 0

In [ ]:
baked[baked[country_qcode].str.contains("France")].reset_index()

In [ ]:
results['Solomon Island']

In [ ]:
with open('./results/transformed-' + DATA_SOURCE.replace('csv','json'), 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
## INPUT CONNECTION

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import numpy as np
from app.models import Countries, Groups, Values, CountryGroups, CountryValues
from app.connection import engine_url, write_data

In [ ]:
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()

In [ ]:
values = forms[['q','s','t']].rename(columns={'q':'description','s':'name','t':'type'})
values['code'] = values['description'].apply(lambda x:x.split(' ')[0].replace('.','').upper())
values = values[values['name'] != COUNTRY_QUESTION].reset_index()
for value in values.to_dict('records'):
    value['parent_id'] = None
    value['units'] = None
    input_data = Values(value)
    write_data(session, input_data, value, False)

In [ ]:
saved_values = pd.read_sql_table('values', engine_url)

In [ ]:
child_values = forms[forms['t'] == 'OPTIONS'].reset_index()
child_values['code'] = child_values['q'].apply(lambda x:x.split(' ')[0].replace('.','').upper())

In [ ]:
saved_values = saved_values.merge(child_values, on='code', how='left')
option_values = saved_values[saved_values['type'] == 'OPTIONS']
option_values = option_values[['id','type','code','units','description','a','color']].to_dict('records')

In [ ]:
for option_value in option_values:
    for index, child in enumerate(option_value['a']):
        child_value = {
            'parent_id': option_value['id'],
            'units': option_value['units'],
            'code': option_value['code'] + '#' + str(index),
            'description': child['v'] + ' - ' + option_value['description'],
            'type': 'OPTION',
            'name': child['s']
        }
        input_data = Values(child_value)
        write_data(session, input_data, child_value, False)

In [ ]:
unep_countries = pd.read_csv('./source/country_shapefile.csv')
unep_countries = unep_countries.rename(columns={'Terr_Name':'name','STATUS':'status'})

In [ ]:
countries = unep_countries[unep_countries['status'] == 'Member State'].rename(columns={'ISO3_CODE':'code'})
countries = countries[['code','name','status']].to_dict('records')
for country in countries:
    input_data = Countries(country)
    write_data(session, input_data, country, False)

In [ ]:
data_groups = pd.read_csv('./source/list_of_regional_seas_country.csv')
data_groups = data_groups[['Group','Country']].rename(columns={'Country':'name','Group':'group'})

In [ ]:
# DEBUG GROUPS
countries = unep_countries[unep_countries['status'] == 'Member State'].rename(columns={'ISO3_CODE':'code'})
country_groups = countries.merge(data_groups, on='name', how='left', indicator=True)
groups = country_groups[country_groups['_merge'] == 'both'].groupby('group').first().reset_index()['group'].to_frame('name')
groups['parent_id'] = None

In [ ]:
import wikipedia

In [ ]:
def wiki(search):
    description = None
    #try:
    #    description = wikipedia.summary(search, sentences=1)
    #except:
    #    pass
    return description

def groupcode(x):
    x = x.replace('/',' ').replace('(','').replace(')','').split(' ')
    code = ''
    if len(x) == 1:
        x = x[0][0:3]
    for y in x:
        code += y[0].upper()
    return code

In [ ]:
groups['description'] = groups['name'].apply(lambda x:wiki(x))
groups['code'] = groups['name'].apply(lambda x: groupcode(x))

In [ ]:
for group in groups.to_dict('records'):
    input_data = Groups(group)
    write_data(session, input_data, group, False)

In [ ]:
error_country = pd.DataFrame(countrylist)
error_country = error_country[['v']]
error_country = error_country.rename(columns={'v':'name'})
error_country = error_country.merge(unep_countries, on='name', how='left', indicator=True)
error_country = error_country[error_country['ISO3_CODE'] != error_country['ISO3_CODE']]
error_country.to_csv('./results/error-combined_source-and-shapefile.csv')

In [ ]:
saved_groups = pd.read_sql_table('groups', engine_url)
country_groups = country_groups.rename(columns={'name':'country','group':'name','id':'group_id'})
country_groups = country_groups[country_groups['_merge'] == 'both'][['name','country']]
saved_groups = saved_groups.merge(country_groups, on='name', how='left', indicator=True).rename({'id':'group_id'})

In [ ]:
saved_groups

In [ ]:
saved_countries = pd.read_sql_table('countries', engine_url).rename(columns={'name':'country','id':'country_id'})
saved_countries = saved_countries[['country','country_id']]

In [ ]:
saved_countries

In [ ]:
country_groups = saved_groups.merge(saved_countries, on='country',how='left')
country_groups = country_groups.rename(columns={'id':'group_id'})
country_groups = country_groups[['group_id','country_id']].to_dict('records')

In [ ]:
for country_group in country_groups:
    input_data = CountryGroups(country_group)
    write_data(session, input_data, country_group, False)

In [ ]:
for res in results:
    country = session.query(Countries).filter(Countries.name == res).first()
    data = {}
    if country:
        for question in results[res]:
            for answer in results[res][question]:
                option = session.query(Values).filter(Values.code == answer).first()
                country_value = {
                    'country_id':country.id,
                    'value_id':option.id,
                    'value':results[res][question][answer],
                    'description': None
                }
                input_data = CountryValues(country_value)
                write_data(session, input_data, country_value, False)

In [ ]:
results